In [76]:
%pylab
from px import *
import numpy as np

class log(object):
    def __init__(self,logname):
        
        with open(logname, 'r') as f:
             self.lines = f.readlines()
                
        self.invalid = []
        self.rsp = []
        self.cmd = []
        self.other = []
        self.spurious = []
        self.all = []
        
        lno = 1
        for line in self.lines:
            self.parse_line(line,lno)
            lno += 1
            
        self.headers()
            
    def parse_line(self,line,lno):
        
        if "sending" in line:
            self.cmd.append(self.parse_cmd_rsp(line,lno))
            self.all.append("cmd")
        elif "receiving" in line:
            self.rsp.append(self.parse_cmd_rsp(line,lno))
            self.all.append("rsp")
        elif "invalid" in line:
            self.invalid.append(lno)
            self.all.append("invalid")
        elif "Spurious" in line:
            self.spurious.append(lno)
            self.all.append("spurious")
        else:
            self.other.append(lno)
            self.all.append("other")
            
        _all = np.array(self.all)
            
        self.spurious_flag = _all == "spurious"
        self.invalid_flag = _all == "invalid"
        
        self.spurious_rate = np.sum(self.spurious_flag) / float(len(_all))
        self.invalid_rate = np.sum(self.invalid_flag) / float(len(_all))
        
    def parse_cmd_rsp(self,line,lno):
            
        l = line.split(' ')
        t = float(l[1][:-3])
        val = ''
        for v in l[2:]:
            if len(v) != 2:
                v = '0'+v
            val += v
                
        return lno,t,command(val)
        
    def headers(self):
        
        self.cmd_header = np.array([c[2].header for c in self.cmd])
        self.rsp_header = np.array([c[2].header for c in self.rsp])
        
        self.cmd_time = np.array([c[1] for c in self.cmd])
        self.rsp_time = np.array([c[1] for c in self.rsp])
        
        self.cmd_lno = np.array([c[0] for c in self.cmd])
        self.rsp_lno = np.array([c[0] for c in self.rsp])

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [77]:
log95 = log("log_RP95Hz.txt")
log80 = log("log_RP80Hz.txt")
log59 = log("log_RP59Hz.txt")

In [24]:
plot( np.diff(cmd95_lno) )

In [58]:
ylabel("Command Header Value")
xlabel("Command Number")

plot( log95.cmd_header, label = "95 Hz",linewidth=2 )
plot( log80.cmd_header, label = "80 Hz",linewidth=2 )
plot( log59.cmd_header, label = "59 Hz",linewidth=2 )

legend()

In [64]:
ylabel("Response Header Value")
xlabel("Response Number")

plot( log95.rsp_header, label = "95 Hz",linewidth=2 )
plot( log80.rsp_header, label = "80 Hz",linewidth=2 )
plot( log59.rsp_header, label = "59 Hz",linewidth=2 )

legend()

In [74]:
ylabel("d cmd position / d line number")
xlabel("Line Number")

s = 200
e = 700

#plot( np.diff(log95.cmd_lno)[s:e], label = "95 Hz",linewidth=2, color = 'b' )
#plot( np.diff(log80.cmd_lno)[s:e], label = "80 Hz",linewidth=2, color = 'g' )
plot( np.diff(log59.cmd_lno)[s:e], label = "59 Hz",linewidth=2, color = 'r' )

legend()

In [18]:
plot(np.diff(log95.cmd_time))

In [19]:
plot(log95.cmd_time)

In [20]:
plot(log59.cmd_time)

In [35]:
plot(log95.spurious_flag)

In [75]:
scatter(np.arange(len(log95.all)),log95.invalid_flag,color="b")
scatter(np.arange(len(log80.all)),log80.invalid_flag,marker="+",color="g")
scatter(np.arange(len(log59.all)),log59.invalid_flag,marker="+",color="r")

In [50]:
np.sum(log59.spurious_flag) / float(len(log59.spurious_flag))

0.031350681536555146

In [49]:
np.sum(log80.spurious_flag) /float(len(log80.spurious_flag))

0.041534054742202418

In [48]:
np.sum(log95.spurious_flag) / float(len(log95.spurious_flag))

0.1117774046533941

In [62]:
log95.rsp[0]

(1781, 0.359546, <px.command at 0x6026048>)

In [63]:
int("a5bb",16)

42427

In [80]:
log80.spurious_rate

0.041534054742202418

In [86]:
plot(np.diff(log95.cmd_lno))
plot(log95.invalid_flag)
plot(log95.spurious_flag)